In [1]:
import torch
import torchvision
from torch import nn 
#from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import tqdm
import scipy.io as io
import numpy as np
import sys
sys.path.append('../')
import dataset_utils

In [2]:
beetle_dataset = io.loadmat("standard_beetle.mat")
beetle_dataset.keys()
DAT = beetle_dataset['DAT'][0]
boldids = []
for boldid in np.squeeze(DAT[5][0]):
    boldids.append(boldid[0])
nucleotides = []
for nucleotide in DAT[0].squeeze():
    #print(len(nucleotide[0]))
    nucleotides.append(nucleotide[0])

In [3]:
onehot_nucleotides = torch.tensor(np.array([dataset_utils.one_hot_encoding(x) for x in nucleotides]))

In [50]:
#all_dnas = torch.load('../tensor_dataset/all_dnas.pt')

In [6]:
bb = dict()
bb['onehots'] = onehot_nucleotides.numpy()
io.savemat('onehots_beetles.mat',bb)

In [56]:
class DNAdataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.float()
        self.targets = None
        
    def __getitem__(self, index):
        x = self.data[index].unsqueeze(0)
        return x, -1
    def __len__(self):
        return len(self.data)
        
dataset_nucleotides = DNAdataset(onehot_nucleotides, None)


In [57]:
dataloader_nucleotides = DataLoader(dataset_nucleotides, batch_size=32,shuffle=False)


In [58]:
class TinyModel(torch.nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = torch.nn.Conv2d(1,16,(5,1))
        self.activation1 = torch.nn.LeakyReLU()
        self.norm1 = torch.nn.BatchNorm2d(16)
        self.conv2 = torch.nn.Conv2d(16,1,(5,1))
        self.activation2 = torch.nn.LeakyReLU()
        self.norm2 = torch.nn.BatchNorm2d(1)
        self.dropout1= torch.nn.Dropout(0.70)
        self.flat = torch.nn.Flatten()
        self.linear = torch.nn.Linear(3250,1500)
        self.dropout2= torch.nn.Dropout(0.70)
        self.activation3 = torch.nn.LeakyReLU()
        self.linear2 = torch.nn.Linear(1500,1050)
        #self.softmax = torch.nn.Softmax()
    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.activation2(x)
        x = self.norm2(x)
        x = self.dropout1(x)
        x = self.flat(x)
        x = self.linear(x)
        x = self.dropout2(x)
        x = self.activation3(x)
        x = self.linear2(x)
        return x
    def feature_extract(self,x):
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.flat(x)
        return x




model = TinyModel()
model.cuda()
 
optimizer = torch.optim.Adam(tinymodel.parameters(),weight_decay=1e-5)
n_params = dataset_utils.count_trainable_parameters(tinymodel);
print(n_params)

6452761


In [59]:
state_dict = torch.load('../checkpoints/CNN_DNA_weights_for_unseen')
model.load_state_dict(state_dict['model_state_dict'])
optimizer.load_state_dict(state_dict['optimizer_state_dict'])

In [60]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
with torch.no_grad():
    features = []
    labels = np.array([]) 
    for dnas,batch_labels in dataloader_nucleotides:
        #print(dnas.shape)
        dnas = dnas
        dnas = dnas.to(device)
        fts = model.feature_extract(dnas)
        labels = np.concatenate((labels, batch_labels.cpu().numpy()))
        features.append(fts.cpu().numpy())
        torch.cuda.empty_cache()
    features = torch.tensor(np.concatenate(features))
    labels = torch.tensor(labels)


tensor(1.1578)

# Save extracted features in .mat

In [66]:
beetle_dataset = dict()
beetle_dataset['all_dna_features_cnn_new'] = features
io.savemat('beetle_features.mat',beetle_dataset)


# Save CNN weights

In [13]:
torch.save({
            'epoch':50,
            'model_state_dict': tinymodel.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "checkpoints/CNN_DNA_weights_for_unseen")

# Random Forest (only used to try features, not indicative of final results)

In [14]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(expanded_train_dna_features,expanded_train_dna_labels )

In [15]:
train_predicted_labels = clf.predict(expanded_train_dna_features)
print(f"Training species accuracy:\
{np.count_nonzero(train_predicted_labels==expanded_train_dna_labels.numpy())/len(expanded_train_dna_labels)}")

Training species accuracy:0.9993864560165657


In [16]:
val_predicted_labels= clf.predict(expanded_val_dna_features)
print(f"Validation species accuracy:{np.count_nonzero(val_predicted_labels==expanded_val_dna_labels.numpy())/len(expanded_val_dna_labels)}")

Validation species accuracy:0.4567936736161035


In [17]:

temp_val_predicted_probs = clf.predict_proba(expanded_val_dna_features)
val_predicted_probs = np.zeros((len(temp_val_predicted_probs),1050))
for i, cls in enumerate(np.arange(1050)):
    if cls in clf.classes_:
        val_predicted_probs[:, cls] = temp_val_predicted_probs[:, list(clf.classes_).index(cls)]
import math
n_correct_genus = 0
for i in range(len(expanded_val_dna_labels)):
    #label_best_specie = val_predicted_probs[i].argmax()
    label_best_specie = val_predicted_labels[i]
    assert(val_predicted_labels[i]==val_predicted_probs[i].argmax())
    #print(label_best_specie.item())
    genus_of_best_species = species2genus[int(label_best_specie.item())]
    #species_same_genus = [k for k,v in species2genus.items() if v == genus_of_best_species]
    #reduced_species = val_predicted_probs[i][species_same_genus]
    #normalized_reduced_species = reduced_species/(reduced_species.sum())
    
    real_genus = species2genus[int(expanded_val_dna_labels[i].item())]
    predicted_genus = genus_of_best_species
    if real_genus == predicted_genus:
        n_correct_genus+=1
print(f"Validation genus accuracy: {n_correct_genus/len(expanded_val_dna_labels)}")


Validation genus accuracy: 0.8046010064701653
